In [1]:
import pandas as pd
import spacy
import string
import regex as re
import nltk
import ast
import copy
import glob
# import advertools as adv
import plotly.graph_objects as go


pd.options.display.max_colwidth = 285



from textblob import TextBlob

# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


from spacymoji import Emoji
from nltk.corpus import stopwords
from urllib.parse import urlparse
from textblob import Word
# from sklearn.metrics.pairwise import cosine_similarity



# nlp = spacy.load("en_core_web_sm")
# emoji = Emoji(nlp)
# nlp.add_pipe("emoji", first=True)


# stemmer = nltk.SnowballStemmer("english")

/home/niloofar/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
%run ./modules/load_data_module.ipynb
%run ./modules/extract_info_clean_data_module.ipynb
%run ./modules/update_dataframe_module.ipynb
# %run ./modules/sentiment_module.ipynb
# %run ./modules/semantic_similarity_module.ipynb

In [3]:
sample_tweets = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_tweets_final_version.feather')
sample_replies = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_replies_final_version.feather')
sample_quotes = pd.read_feather('/home/niloofar/Developement/Climate-Change/data/sample_data/sample_quotes_final_version.feather')

In [4]:
print(len(sample_tweets), len(sample_replies), len(sample_quotes))

3976 9625 2074


In [5]:
def add_cs_score_column(df):
    df['GS_controversiality_score'] = None
    return df

In [11]:
sample_tweets.head(1)

,id,author_id,conversation_id,created_at,lang,retweet_count,reply_count,like_count,quote_count,reply_settings,...,avg_conversation_sentiment,var_replies_sentiment,var_quotes_sentiment,var_conversation_sentiment,pct_similar_sentiment_replies,pct_similar_sentiment_quotes,clean_text,var_replies_semantic,var_quotes_semantic,var_conversation_semantic
0,1553749453990817792,1421531202037686272,1553749453990817792,2022-07-31T14:28:31.000Z,en,1,0,0,0,everyone,...,0.0,0.0,0.0,0.0,0.0,0.0,this biblical scale disinformation mind game create panic destroy civilization zaidhamid globalreset wef nwo foodsecurity energycrisis buildbackbetter globalwarming haarp weatherupdate,0.0,0.0,0.0


In [9]:
sample_tweets['var_conversation_sentiment'].tolist()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.020833333333333336,
 0.0,
 0.15078254564102564,
 0.0,
 0.0,
 0.0,
 0.016928,
 0.03339228000000001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.13035618000000002,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.047196840000000004,
 0.0,
 0.0,
 0.0,
 0.28366188916666674,
 0.0,
 0.0,
 0.1500366928349944,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07789716492647059,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.31904071999999994,
 0.0,
 0.19815634792569656,
 0.0,
 0.09772629288043479,
 0.0,
 0.024354245,
 0.0,
 0.0,
 0.042457835833333325,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2103409799999999

In [ ]:
def update_controversiality_score(df_tweet, df_reply, df_quote, from_idx, to_idx):
    """
    input:
        df_tweet: sample dataframe of tweets
        df_reply: sample dataframe of replies
        df_quote: sample dataframe of quotes
        from_idx: integer, the first index of the tweets sub-sample. 
        to_idx: integer, the last index of the tweets sub-sample

    output:
        dataframe, the tweets dataframe, the GS_controversiality_score of the selected sub-sample is updated.
    
    How it works?
        it shows a tweet and all information about it as a dataframe
        it then asks if you want to see the dataframe contining the replies and the information about replies
        TO SEE THE REPLIES: press y, to continue without reading the replies, press any other key
        it ask it you want to see the dataframe containing the quotes
        TO SEE THE QUOTES: press y, to continue without reading the replies press any other key
        
        Finally:
            it asks you to give some score for the GS_controversiality_score. the inputs are:
                0 : not controversial
                1 : somewhat controversial
                2 : controversial
                3 : very controversial
    """
    df_sample = copy.deepcopy(df_tweet.iloc[from_idx:to_idx])
    
    for i, row in df_sample.iterrows():
        display(row.to_frame())
        
        print('Read replies?')
        reply = input()
        
        if reply == 'y':
            display(get_conversation(row['conversation_id'], df_reply))
            
        print('Read quotes?')
        quote = input()
        
        if quote == 'y':
            display(get_conversation(row['conversation_id'], df_reply))
            
            
        print('The score would be:')
        score_range = [0, 1, 2, 3]
        score = input()
        
        if int(score) not in score_range:
            print('INVALID SCORE')
            score = input()
            if int(score) not in score_range:
                print("FINAL CHANCE TO GIVE A VALID SCORE!")
                score = input()
        
        
        df_tweet.at[i, ['GS_controversiality_score']] = score
        
        print(f"score stored for the {i} row is {score}")
        
            
        print('********************************************************************************************', end="")
       
        
    return df_tweet
        
        


In [ ]:
# Toy example of the function
df = update_controversiality_score(sample_tweets, sample_replies, sample_quotes, 1, 3)
df


 
<span style="color:red"> **To save the scores use  df.to_csv('name.csv', index=False)**</span>